# LaunchPad BioMedical Robotics Workshop
# Module 3 - Scrubbing with servomotor

Connect power, ground from Servo pins on shield to Black and Red wires on scrubber's servo
Connect pin 14 on shield to White wire of scrubber servo

## Functions for controlling the Servos

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
import time
import math


def read_i2c_dev_reg(i2c_master, device_addr, reg_addr, num_bytes):
    """
    Read an I2C device's memory.
    
    :param i2c_master: the I2C Master device (Arduino MicroBlaze)
    :param device_addr: the I2C Node Address (accessory device)
    :param reg_addr: the register address on the I2C node to read
    :param num_bytes: the number of bytes to read
    """
    i2c_buffer = bytearray(num_bytes + 1)
    i2c_buffer[0] = reg_addr
    i2c_master.write(device_addr, i2c_buffer, 1)  # write the address
    i2c_master.read(device_addr, i2c_buffer, num_bytes)  # read the bytes back
    return i2c_buffer[0:num_bytes]


def write_i2c_dev_reg(i2c_master, device_addr, reg_addr, new_value, num_bytes):
    """
    Write to an I2C device's memory.
    
    :param i2c_master: the I2C Master device (Arduino MicroBlaze)
    :param device_addr: the I2C Node Address (accessory device)
    :param reg_addr: the register address on the I2C node to read
    :param new_value: the new value to write to the register
    :param num_bytes: the number of bytes to write
    """
    i2c_buffer = bytearray(num_bytes + 1)
    i2c_buffer[0] = reg_addr
    for idx in range(num_bytes, 0, -1):
        i2c_buffer[idx] = new_value >> (8 * (idx - 1)) & 0xFF  # restrict to one byte
    i2c_master.write(device_addr, i2c_buffer, num_bytes + 1)  # write the address
    return i2c_buffer


def write_i2c_dev_reg_bytes(i2c_master, device_addr, reg_addr, new_bytes):
    """
    Write to an I2C device's memory.
    
    :param i2c_master: the I2C Master device (Arduino MicroBlaze)
    :param device_addr: the I2C Node Address (accessory device)
    :param reg_addr: the register address on the I2C node to read
    :param new_bytes: the new bytearray to write to the memory
    """
    num_bytes = len(new_bytes)
    i2c_buffer = bytearray(num_bytes + 1)
    i2c_buffer[0] = reg_addr
    for idx, byte in enumerate(new_bytes):
        i2c_buffer[idx+1] = byte
    i2c_master.write(device_addr, i2c_buffer, num_bytes + 1)  # write the address
    return i2c_buffer


def amfs_set_pwm_freq(i2c_master, i2c_addr=0x60, pwm_freq=1600) -> bool:
    """
    Set the PWM output frequency coming out of the AFMS.
    
    :param i2c_master: the I2C Master device (Arduino MicroBlaze)
    :param i2c_addr: the Adafruit Motor Shield I2C address
    :param pwm_freq: the base square wave frequency of the PWM output
    """
    pwm_freq_fixed = pwm_freq * 0.9  # due to overshoot problem
    prescaleval = math.floor(25000000 / 4096 / pwm_freq - 1 + 0.5)
    oldmode = read_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, 1)[0]
    newmode = (oldmode & 0x7F) | 0x10
    write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, newmode, 1)
    write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0xFE, prescaleval, 1)
    write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, oldmode, 1)
    time.sleep(5e-3)
    write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, oldmode | 0xa1, 1)
    result = read_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0xFE, 1)
    return result == prescaleval


AFMS_MOTOR_PINS = {
    "M1": {"PWM": 8, "in2": 9, "in1": 10},
    "M2": {"PWM": 13, "in2": 12, "in1": 11},
    "M3": {"PWM": 2, "in2": 3, "in1": 4},
    "M4": {"PWM": 7, "in2": 6, "in1": 5},
}

def afms_set_pin_starts(
    pin_set: int, 
    on_start: int, 
    off_start: int, 
    i2c_master, 
    i2c_addr=0x60) -> bool:
    """
    Set the pin output counts for when the ON (high) and OFF (low) periods of the PWM signals should start.

    :param pin_set: the pin to set the counts for
    :param on_start: the value of the on counts
    :param off_start: the value of the off counts
    :param i2c_master: the i2c controller object
    :param i2c_addr: the i2c address of the motor shield PCA9685
    :returns: True if command was successful
    """
    pca_pin_reg_base = 0x06
    pca_pin_reg_addr = pca_pin_reg_base + 4 * pin_set
    pca_pin_vals = bytearray(4)
    pca_pin_vals[0] = on_start & 0xFF
    pca_pin_vals[1] = (on_start >> 8) & 0xFF
    pca_pin_vals[2] = off_start & 0xFF
    pca_pin_vals[3] = (off_start >> 8) & 0xFF
    write_i2c_dev_reg_bytes(i2c_master, i2c_addr, pca_pin_reg_addr, pca_pin_vals)
    result = read_i2c_dev_reg(i2c_master, i2c_addr, pca_pin_reg_addr, 4)
    return result == pca_pin_vals

def afms_set_pin_on(pin_set: int, i2c_master, i2c_addr=0x60) -> bool:
    afms_set_pin_starts(pin_set, 4096, 0, i2c_master, i2c_addr)

def afms_set_pin_off(pin_set: int, i2c_master, i2c_addr=0x60) -> bool:
    afms_set_pin_starts(pin_set, 0, 0, i2c_master, i2c_addr)


def servo_angle(theta: int, theta_min_ms: float, theta_max_ms: float, servo_angle_max: int, servo_pin: int, pwm_base_freq: int, i2c_master, i2c_addr=0x60) -> None:    
    pwm_base_ms = round((1 / pwm_base_freq) * 1000)
    pwm_val_min_angle = (theta_min_ms / pwm_base_ms) * 4095
    pwm_val_max_angle = (theta_max_ms / pwm_base_ms) * 4095
    scaled_theta = (theta / servo_angle_max) * (pwm_val_max_angle - pwm_val_min_angle) + pwm_val_min_angle
    afms_set_pin_starts(servo_pin, 0, round(scaled_theta), i2c_master, i2c_addr)
    print(f"Servo value = {scaled_theta}")


## Setting up the motor shield to run the servo

In [ ]:
base = BaseOverlay('base.bit')  # load overlay into the Programmable Logic FPGA portion
lib = MicroblazeLibrary(base.ARDUINO, ['i2c'])  # load the i2c driver using the ARDUINO MicroBlaze IO Processor

lib.i2c_get_num_devices()

i2c_master = lib.i2c_open_device(0)

I2C_ADDRESS_AFMS = 0x60

""" Check for default value of 0x04 in reg 0x01 """
result = read_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x01, 1)
# print(result.hex())

""" Enable register auto-increment """
result = write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, 0x20, 1)
# print(result.hex())

""" Check for register auto-increment, value of 0x20 in reg 0x00 """
result = read_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, 1)
# print(result.hex())

# Servos operate at 50 Hz; 
# Stepper just needs to account for this 20 ms base period (aka, each step will require 20 ms, so time for full rotation is ~4 seconds (200 steps))
amfs_set_pwm_freq(i2c_master, I2C_ADDRESS_AFMS, 50)  

## Running the servo

Run the cell below.  Then, what can you change in the cell below to help you answer the questions in the workbook?

In [ ]:
for angle in range(0, 190, 10):
    servo_angle(
        theta=angle,
        theta_min_ms=0.7,
        theta_max_ms=2.45,
        servo_angle_max=180,
        servo_pin=14,
        pwm_base_freq=50,
        i2c_master=i2c_master)
    print(angle)
    time.sleep(0.1)